In [1]:
import numpy as np
import pandas as pd

In [12]:
df=pd.read_csv('data\BTC-USD.csv')
df_Indicators=pd.read_csv('data\BTC-USD.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014/10/1,387.427002,391.378998,380.779999,383.614990,383.614990,2.622940e+07
1,2014/10/2,383.988007,385.497009,372.946014,375.071991,375.071991,2.177770e+07
2,2014/10/3,375.181000,377.695007,357.859009,359.511993,359.511993,3.090120e+07
3,2014/10/4,359.891998,364.487000,325.885986,328.865997,328.865997,4.723650e+07
4,2014/10/5,328.915985,341.800995,289.295990,320.510010,320.510010,8.330810e+07
...,...,...,...,...,...,...,...
3068,2023/2/24,23946.007810,24103.705080,23007.072270,23198.126950,23198.126950,2.681174e+10
3069,2023/2/25,23200.125000,23210.210940,22861.558590,23175.375000,23175.375000,1.610072e+10
3070,2023/2/26,23174.150390,23654.367190,23084.220700,23561.212890,23561.212890,1.664453e+10
3071,2023/2/27,23561.451170,23857.890630,23205.878910,23522.871090,23522.871090,2.266076e+10


In [3]:
def SMA(close,n):   
    return close.rolling(window=n).mean()

In [4]:
def WMA(close, n):
    weights = np.array(range(1, n+1))
    sum_weights = np.sum(weights)

    res = close.rolling(window=n).apply(lambda x: np.sum(weights*x) / sum_weights, raw=False)
    return res

In [5]:
def EMA(close, n):    
    res=close.ewm(span=n,min_periods=n).mean()
    return res

In [6]:
def RSI(close, n):
    # RSI
    change = close.diff(1)
    df["Gain"] = change.mask(change < 0, 0)
    df["Loss"] = abs(change.mask(change > 0, 0))
    df["AVG_Gain"] = df.Gain.rolling(n).mean()
    df["AVG_Loss"] = df.Loss.rolling(n).mean()
    df["RS"] = df["AVG_Gain"] / df["AVG_Loss"]
    df["RSI"] = 100 - (100 / (1 + df["RS"]))
    return df["RSI"]


In [7]:
def Stoch_RSI(close, n):
    #Stochastic RSI
    RSI=df["RSI"]
    LL_RSI = RSI.rolling(n).min()
    HH_RSI = RSI.rolling(n).max()         
    return (RSI - LL_RSI) / (HH_RSI - LL_RSI)

In [8]:
def BB(close, n):
    MA = pd.Series(close.rolling(n).mean())
    STD = pd.Series(close.rolling(n).std())
    bb1 = MA + 2 * STD
    df["Upper Bollinger Band"] = pd.Series(bb1)
    bb2 = MA - 2 * STD
    df["Lower Bollinger Band"] = pd.Series(bb2)
    return df["Upper Bollinger Band"],df["Lower Bollinger Band"]

In [9]:
def MACD(close):
    df['EMA12'] = close.ewm(span=12, adjust=False).mean() 
    df['EMA26'] = close.ewm(span=26, adjust= False).mean() 
    return df['EMA12'] - df['EMA26'] 


In [13]:
df_Indicators['SMA_7']=SMA(df['Adj Close'],7)
df_Indicators['SMA_14']=SMA(df['Adj Close'],14)

df_Indicators['EMA_7']=EMA(df['Adj Close'],7)
df_Indicators['EMA_14']=EMA(df['Adj Close'],14)

df_Indicators['RSI']=RSI(df['Adj Close'],14)


df_Indicators['Stoch_RSI']=Stoch_RSI(df['Adj Close'],14)

df_Indicators['Upper_BB']=BB(df['Adj Close'],20)[0]
df_Indicators['Lower_BB']=BB(df['Adj Close'],20)[1]

df_Indicators['MACD']=MACD(df['Adj Close'])
df_Indicators= df_Indicators.dropna()
df_Indicators

,Date,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD
27,2014/10/28,353.214996,359.984009,352.678986,357.618011,357.618011,7.845880e+06,358.928859,373.135500,360.189252,365.640480,26.597270,0.000000,407.000618,341.261085,-4.591695
28,2014/10/29,357.088989,357.833008,335.342987,335.591003,335.591003,1.819270e+07,352.133575,368.908214,354.038225,361.569705,22.686407,0.000000,409.625952,335.692252,-6.585445
29,2014/10/30,335.709015,350.912994,335.071991,345.304993,345.304993,3.017790e+07,350.260433,366.247428,351.854527,359.371035,32.401375,0.170521,410.516219,333.176283,-7.297547
30,2014/10/31,345.009003,348.045013,337.141998,338.321014,338.321014,1.254540e+07,347.399863,363.001929,348.470696,356.530731,29.646025,0.122158,411.960112,329.334590,-8.329426
31,2014/11/1,338.649994,340.528992,321.054993,325.748993,325.748993,1.667720e+07,344.325291,358.309572,342.789699,352.383942,21.806794,0.000000,413.709058,322.305642,-10.045852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068,2023/2/24,23946.007810,24103.705080,23007.072270,23198.126950,23198.126950,2.681174e+10,24224.126396,23553.841937,23885.895833,23687.895916,61.096623,0.844457,25484.342390,21030.005073,568.123223
3069,2023/2/25,23200.125000,23210.210940,22861.558590,23175.375000,23175.375000,1.610072e+10,24014.711776,23647.020509,23708.265624,23619.559794,59.629588,0.803424,25491.229681,21045.088680,479.526729
3070,2023/2/26,23174.150390,23654.367190,23084.220700,23561.212890,23561.212890,1.664453e+10,23905.221820,23773.664063,23671.502441,23611.780207,62.527341,0.856062,25521.588973,21094.839739,435.427920
3071,2023/2/27,23561.451170,23857.890630,23205.878910,23522.871090,23522.871090,2.266076e+10,23718.610770,23896.147601,23634.344603,23599.925658,62.084345,0.831206,25536.457725,21105.828994,392.856825


In [14]:
df_Indicators.to_csv('BTC-Indicators.csv')